#  Direcotry

In [ ]:
ls

In [2]:
cd dataset

C:\Users\12266\DATATHON\dataset


In [ ]:
ls Crashes

In [ ]:
ls "Traffic, Investigations _ Other"

In [ ]:
ls "Data Open 2023 Additional Data"

In [49]:
#import related packages
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np

# Load Data
## Police District

In [50]:
df_police_districts = pd.read_csv("Traffic, Investigations _ Other/police_districts.csv")
df_police_districts.head()

,OBJECTID,PERIMETER,DISTRICT_NUM,LOCATION,LAT,LNG,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
0,343,69282.58846,16,39th St. & Lancaster Ave.,39.96158,-75.19940,SWPD,1.216700e+08,1.927226e+07,27575.07918
1,344,33150.15496,17,20th St. & Federal St.,39.93728,-75.17670,SPD,5.786368e+07,9.154950e+06,13179.95335
2,345,54403.93004,18,55th St. & Pine St.,39.95426,-75.23217,SWPD,9.881929e+07,1.564117e+07,21629.61069
3,346,51597.05160,35,N. Broad St. & Champlost St.,40.04442,-75.14351,NWPD,1.544309e+08,2.450685e+07,20554.86914
4,347,58075.01444,39,22nd St. & Hunting Park Ave.,40.01362,-75.15884,NWPD,1.578976e+08,2.503787e+07,23123.58026


## Crime with District

In [51]:
with open('Data Open 2023 Additional Data/philadelphia.geojson') as handle:
    philly_geo = json.loads(handle.read())


crime = pd.read_csv("Traffic, Investigations _ Other/crimes.csv")
district_gdf = gpd.read_file('Data Open 2023 Additional Data/Boundaries_District.geojson')

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(crime['lng'], crime['lat'])]
crime_gdf = gpd.GeoDataFrame(crime, crs="EPSG:4326", geometry=geometry)
crime_with_district = gpd.sjoin(crime_gdf, district_gdf, how='left', op='within')
crime_with_district = crime_with_district.dropna(subset=["DIST_NUM"])
crime_with_district.head(), len(crime_with_district)

(   objectid psa dispatch_date dispatch_time         location_block  \
 0    218891   2    2022-09-08      15:21:00  3200 BLOCK N BROAD ST   
 1    218892   2    2022-09-14      17:41:00  3200 BLOCK N BROAD ST   
 2    218893   2    2022-09-19      15:23:00  3200 BLOCK N BROAD ST   
 3    218894   2    2022-08-22      09:38:00        0 BLOCK WOLF ST   
 4    218895   2    2022-08-20      10:31:00        0 BLOCK WOLF ST   
 
     text_general_code        lat        lng          fips  \
 0              Thefts  40.002741 -75.153068  4.210102e+11   
 1              Thefts  40.002741 -75.153068  4.210102e+11   
 2              Thefts  40.002741 -75.153068  4.210102e+11   
 3  Theft from Vehicle  39.918351 -75.145999  4.210198e+11   
 4  Theft from Vehicle  39.918351 -75.145999  4.210198e+11   
 
                      geometry  ...  DISTRICT_ID  DIST_NUM SUM_AREA  DIST_NUMC  \
 0  POINT (-75.15307 40.00274)  ...         None      39.0     None         39   
 1  POINT (-75.15307 40.00274)  ..

In [11]:
crime_with_district["dispatch_date"].max(),crime_with_district["dispatch_date"].min() 

('2022-12-31', '2012-12-31')

## Crash with District

In [53]:
df_crash = pd.read_csv("Crashes/crash_info_general.csv")

district_gdf = gpd.read_file('Data Open 2023 Additional Data/Boundaries_District.geojson')

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(df_crash['DEC_LONG'], df_crash['DEC_LAT'])]
crash_gdf = gpd.GeoDataFrame(df_crash, crs="EPSG:4326", geometry=geometry)
crash_with_district = gpd.sjoin(crash_gdf, district_gdf, how='left', op='within')
crash_with_district = crash_with_district.dropna(subset=["DIST_NUM"])
print(len(crash_with_district))
crash_with_district.head()

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (97,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


131569


,Unnamed: 0,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,...,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
1,2,2010004457,658.0,2,0,0,0,0,0,0,...,None,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
2,3,2010005005,1217.0,2,0,0,0,0,0,0,...,None,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
4,5,2010005373,36.0,0,0,0,0,0,0,0,...,None,16.0,None,16,39th St. & Lancaster Ave.,686-3160,SWPD,1.216700e+08,1.927226e+07,27575.079183
5,6,2010005374,326.0,1,0,0,0,0,0,0,...,None,15.0,None,15,Harbison Ave. & Levick St.,686-3150,NEPD,3.077864e+08,4.881580e+07,35944.018281
6,7,2010005375,505.0,1,0,0,0,0,0,0,...,None,3.0,None,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793


## Traffic Stop Data

In [52]:
traffic_stops = pd.read_csv("Traffic, Investigations _ Other/traffic_stops_philadelphia.csv")
traffic_stops.head()

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,objectid,date,time,location,lat,lng,district,service_area,subject_age,subject_race,...,outcome,contraband_found,frisk_performed,search_conducted,search_person,search_vehicle,raw_race,raw_individual_contraband,raw_vehicle_contraband,fips
0,1,2014-01-01,01:14:00,NaN,NaN,NaN,19.0,191,31.0,black,...,arrest,True,False,True,True,False,Black - Non-Latino,True,False,NaN
1,2,2014-01-01,01:57:00,NaN,NaN,NaN,12.0,121,21.0,black,...,arrest,False,True,True,True,False,Black - Non-Latino,False,False,NaN
2,3,2014-01-01,03:30:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,24.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
3,4,2014-01-01,03:40:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,20.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
4,5,2014-01-01,08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19.0,193,31.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210101e+11


# Merge Crime, Crash, Traffic Stop

## Deal with crime data

In [16]:
#split crime_with_district into different year_month units
crime_with_district['dispatch_date'] = pd.to_datetime(crime_with_district['dispatch_date'])
crime_with_district['year_month'] = crime_with_district['dispatch_date'].dt.to_period('M')

In [17]:
#### Only the text_general_code will be useful
crime_crosstab = pd.crosstab(index = [crime_with_district['year_month'],crime_with_district['DIST_NUM']], columns = crime_with_district['text_general_code'])
crime_crosstab.columns = ["crime_"+i for i in crime_crosstab.columns]
crime_crosstab['crime_total'] = crime_crosstab.sum(axis=1)
crime_crosstab

crime_Aggravated Assault Firearm  \
year_month DIST_NUM                                     
2012-12    1.0                                      0   
           2.0                                      0   
           3.0                                      0   
           5.0                                      0   
           6.0                                      0   
...                                               ...   
2022-12    25.0                                    37   
           26.0                                    11   
           35.0                                    19   
           39.0                                    16   
           77.0                                     0   

                     crime_Aggravated Assault No Firearm  \
year_month DIST_NUM                                        
2012-12    1.0                                         0   
           2.0                                         0   
           3.0                                         0   
           5.0                                         0   
           6.0                                         0   
...                                                  ...   
2022-12    25.0                                       42   
           26.0                                       14   
           35.0                                       26   
           39.0                                       30   
           77.0                                        0   

                     crime_All Other Offenses  crime_Arson  \
year_month DIST_NUM                                          
2012-12    1.0                              0            0   
           2.0                              0            0   
           3.0                              0            0   
           5.0                              0            0   
           6.0                              0            0   
...                                       ...          ...   
2022-12    25.0                            60            4   
           26.0                            28            1   
           35.0                            59            0   
           39.0                            46            0   
           77.0                             2            0   

                     crime_Burglary Non-Residential  \
year_month DIST_NUM                                   
2012-12    1.0                                    0   
           2.0                                    0   
           3.0                                    0   
           5.0                                    0   
           6.0                                    0   
...                                             ...   
2022-12    25.0                                   7   
           26.0                                  23   
           35.0                                   2   
           39.0                                  12   
           77.0                                   0   

                     crime_Burglary Residential  \
year_month DIST_NUM                               
2012-12    1.0                                0   
           2.0                                0   
           3.0                                0   
           5.0                                0   
           6.0                                0   
...                                         ...   
2022-12    25.0                               9   
           26.0                              28   
           35.0                              16   
           39.0                              20   
           77.0                               0   

                     crime_DRIVING UNDER THE INFLUENCE  \
year_month DIST_NUM                                      
2012-12    1.0                                       0   
           2.0                                       0   
           3.0                                       2   
           5.0       

we conduct the transformation of log(1+x) to all the counting numbers. For counting features, logarithmic transformation is usually a good preprocessing method. This is because count-type features may have a long-tailed distribution, and logarithmic transformation helps reduce the influence of outliers and make the data closer to the normal distribution.

In [110]:
crime_transformed=np.log1p(crime_crosstab)
crime_transformed

crime_Aggravated Assault Firearm  \
year_month DIST_NUM                                     
2012-12    1.0                               0.000000   
           2.0                               0.000000   
           3.0                               0.000000   
           5.0                               0.000000   
           6.0                               0.000000   
...                                               ...   
2022-12    25.0                              1.534194   
           26.0                              1.248441   
           35.0                              1.385227   
           39.0                              1.343703   
           77.0                              0.000000   

                     crime_Aggravated Assault No Firearm  \
year_month DIST_NUM                                        
2012-12    1.0                                  0.000000   
           2.0                                  0.000000   
           3.0                                  0.000000   
           5.0                                  0.000000   
           6.0                                  0.000000   
...                                                  ...   
2022-12    25.0                                 1.560500   
           26.0                                 1.310506   
           35.0                                 1.457646   
           39.0                                 1.489299   
           77.0                                 0.000000   

                     crime_All Other Offenses  crime_Arson  \
year_month DIST_NUM                                          
2012-12    1.0                       0.000000     0.000000   
           2.0                       0.000000     0.000000   
           3.0                       0.000000     0.000000   
           5.0                       0.000000     0.000000   
           6.0                       0.000000     0.000000   
...                                       ...          ...   
2022-12    25.0                      1.631370     0.959135   
           26.0                      1.474144     0.526589   
           35.0                      1.628131     0.000000   
           39.0                      1.579009     0.000000   
           77.0                      0.741276     0.000000   

                     crime_Burglary Non-Residential  \
year_month DIST_NUM                                   
2012-12    1.0                             0.000000   
           2.0                             0.000000   
           3.0                             0.000000   
           5.0                             0.000000   
           6.0                             0.000000   
...                                             ...   
2022-12    25.0                            1.124748   
           26.0                            1.429846   
           35.0                            0.741276   
           39.0                            1.271150   
           77.0                            0.000000   

                     crime_Burglary Residential  \
year_month DIST_NUM                               
2012-12    1.0                         0.000000   
           2.0                         0.000000   
           3.0                         0.000000   
           5.0                         0.000000   
           6.0                         0.000000   
...                                         ...   
2022-12    25.0                        1.194706   
           26.0                        1.474144   
           35.0                        1.343703   
           39.0                        1.397363   
           77.0                        0.000000   

                     crime_DRIVING UNDER THE INFLUENCE  \
year_month DIST_NUM                                      
2012-12    1.0                                0.000000   
           2.0                                0.000000   
           3.0                                0.741276   
           5.0       

## Deal with crash data

In [20]:
number_of_crashes = crash_with_district.groupby("DIST_NUM").count()["CRN"]
number_of_crashes

DIST_NUM
1.0      3333
2.0      6270
3.0      6475
5.0      1999
6.0      6995
7.0      4272
8.0      7706
9.0      4967
12.0     8005
14.0     5866
15.0    12130
16.0     8841
17.0     2537
18.0     7567
19.0     4887
22.0     6135
24.0     6893
25.0     7485
26.0     5183
35.0     7089
39.0     6890
77.0       44
Name: CRN, dtype: int64

In [62]:
#split crash_with_district into different year_month units
crash_with_district['date_str'] = crash_with_district['CRASH_YEAR'].astype(str) + '-' + crash_with_district['CRASH_MONTH'].astype(str).str.zfill(2)
crash_with_district['year_month'] = pd.to_datetime(crash_with_district['date_str']).dt.to_period('M')
crash_with_district = crash_with_district.drop('date_str', axis=1)

crash_with_district['year_month'] 

1         2010-01
2         2010-01
4         2010-01
5         2010-01
6         2010-01
           ...   
133008    2021-07
133009    2021-11
133010    2021-07
133011    2021-11
133012    2021-11
Name: year_month, Length: 131569, dtype: period[M]

We will keep the following features

categorical feat: *_TYPE, ILLUMINATION,  RELATION_TO_ROAD, ROAD_CONDITION, URBAN_RURAL, WEATHER1

numerical feat: *_COUNT of drivers in different age groups &"FATAL_COUNT","SUSP_SERIOUS_INJ_COUNT","SUSP_MINOR_INJ_COUNT"

date feat: YEAR_MONTH

In [147]:
crash_with_district["ARRIVAL_TM"]

1          658.0
2         1217.0
4           36.0
5          326.0
6          505.0
           ...  
133008       NaN
133009       0.0
133010    1000.0
133011       NaN
133012    1541.0
Name: ARRIVAL_TM, Length: 131569, dtype: float64

In [60]:
#combine the age group 16-20
crash_with_district["DRIVER_COUNT_16_20YR"]=crash_with_district[["DRIVER_COUNT_16YR","DRIVER_COUNT_17YR","DRIVER_COUNT_18YR","DRIVER_COUNT_19YR","DRIVER_COUNT_20YR"]].sum(axis=1)
crash_with_district["DRIVER_COUNT_16_20YR"].mean()

0.09879226869551339

In [136]:
manual_cat_list = ["ILLUMINATION", "ROAD_CONDITION", "URBAN_RURAL", "WEATHER1"]
manual_drop_list = ["RDWY_SURF_TYPE_CD", "WORK_ZONE_TYPE","COLLISION_TYPE","TCD_TYPE"]
crash_useful_cat_feat_list = [i for i in crash_with_district.columns if ("_TYPE" in i and i not in manual_drop_list) or i in manual_cat_list]
crash_useful_num_feat_list = ["DRIVER_COUNT_16_20YR","DRIVER_COUNT_50_64YR","DRIVER_COUNT_65_74YR","DRIVER_COUNT_75PLUS","FATAL_COUNT","SUSP_SERIOUS_INJ_COUNT","SUSP_MINOR_INJ_COUNT"]
crash_useful_cat_feat = pd.get_dummies(crash_with_district[crash_useful_cat_feat_list].astype(str))
crash_useful_cat_feat[["DIST_NUM","year_month"]]=crash_with_district[["DIST_NUM","year_month"]]

In [137]:
crash_useful_feat = pd.concat([crash_with_district[crash_useful_num_feat_list], crash_useful_cat_feat], axis=1)
crash_useful_feat = crash_useful_feat.groupby(["year_month","DIST_NUM"]).sum()
# crash_useful_feat['crash_total'] = crash_useful_feat.sum(axis=1)

In [138]:
crash_useful_feat

DRIVER_COUNT_16_20YR  DRIVER_COUNT_50_64YR  \
year_month DIST_NUM                                               
2010-01    1.0                          1                     5   
           2.0                          4                     8   
           3.0                          2                     9   
           5.0                          1                     4   
           6.0                          3                     9   
...                                   ...                   ...   
2021-12    25.0                         3                     8   
           26.0                         4                     9   
           35.0                         4                    24   
           39.0                         2                    13   
           77.0                         0                     2   

                     DRIVER_COUNT_65_74YR  DRIVER_COUNT_75PLUS  FATAL_COUNT  \
year_month DIST_NUM                                                           
2010-01    1.0                          1                    0            0   
           2.0                          4                    1            1   
           3.0                          6                    1            1   
           5.0                          1                    0            0   
           6.0                          2                    1            0   
...                                   ...                  ...          ...   
2021-12    25.0                         3                    0            2   
           26.0                         3                    0            0   
           35.0                         4                    1            0   
           39.0                         1                    0            1   
           77.0                         0                    0            0   

                     SUSP_SERIOUS_INJ_COUNT  SUSP_MINOR_INJ_COUNT  \
year_month DIST_NUM                                                 
2010-01    1.0                            0                     0   
           2.0                            1                     5   
           3.0                            0                     5   
           5.0                            0                     3   
           6.0                            0                     9   
...                                     ...                   ...   
2021-12    25.0                           2                    12   
           26.0                           2                    10   
           35.0                           2                    24   
           39.0                           4                    30   
           77.0                           0                     1   

                     ILLUMINATION_1.0  ILLUMINATION_2.0  ILLUMINATION_3.0  \
year_month DIST_NUM                                                         
2010-01    1.0                     11                 0                 6   
           2.0                     19                 0                17   
           3.0                     18                 2                21   
           5.0                      8                 0                 4   
           6.0                     20                 0                14   
...                               ...               ...               ...   
2021-12    25.0                    17                 0                18   
           26.0                    16                 0                14   
           35.0                    16                 1                26   
           39.0                    17                 0                32   
           77.0                     0                 0                 1   

                     ...  WEATHER1_2  WEATHER1_3  WEATHER1_4  WEATHER1_5  \
year_month DIST_NUM  ...                                                   
2010-01    1.0       ...           0          14           0         

In [139]:
crash_useful_feat.columns 

Index(['DRIVER_COUNT_16_20YR', 'DRIVER_COUNT_50_64YR', 'DRIVER_COUNT_65_74YR',
       'DRIVER_COUNT_75PLUS', 'FATAL_COUNT', 'SUSP_SERIOUS_INJ_COUNT',
       'SUSP_MINOR_INJ_COUNT', 'ILLUMINATION_1.0', 'ILLUMINATION_2.0',
       'ILLUMINATION_3.0', 'ILLUMINATION_4.0', 'ILLUMINATION_5.0',
       'ILLUMINATION_6.0', 'ILLUMINATION_8.0', 'ILLUMINATION_9.0',
       'ILLUMINATION_nan', 'INTERSECT_TYPE_0', 'INTERSECT_TYPE_1',
       'INTERSECT_TYPE_10', 'INTERSECT_TYPE_11', 'INTERSECT_TYPE_12',
       'INTERSECT_TYPE_2', 'INTERSECT_TYPE_3', 'INTERSECT_TYPE_5',
       'INTERSECT_TYPE_6', 'INTERSECT_TYPE_7', 'INTERSECT_TYPE_9',
       'LOCATION_TYPE_0', 'LOCATION_TYPE_1', 'LOCATION_TYPE_2',
       'LOCATION_TYPE_3', 'LOCATION_TYPE_4', 'LOCATION_TYPE_5',
       'LOCATION_TYPE_6', 'LOCATION_TYPE_7', 'LOCATION_TYPE_8',
       'LOCATION_TYPE_99', 'ROAD_CONDITION_0', 'ROAD_CONDITION_1',
       'ROAD_CONDITION_2', 'ROAD_CONDITION_3', 'ROAD_CONDITION_4',
       'ROAD_CONDITION_5', 'ROAD_CONDITION_6', '

In [140]:
#compute the Traffic Hazard Impact Index
w1=3 #subjectively
w2=2
w3=1
crash_useful_feat["index"]=w1*crash_useful_feat["FATAL_COUNT"]+\
                    w2*crash_useful_feat["SUSP_SERIOUS_INJ_COUNT"]+w3*crash_useful_feat["SUSP_MINOR_INJ_COUNT"]
crash_useful_feat = crash_useful_feat.drop(["FATAL_COUNT","SUSP_SERIOUS_INJ_COUNT","SUSP_MINOR_INJ_COUNT"], axis=1)

In [141]:
crash_useful_feat

DRIVER_COUNT_16_20YR  DRIVER_COUNT_50_64YR  \
year_month DIST_NUM                                               
2010-01    1.0                          1                     5   
           2.0                          4                     8   
           3.0                          2                     9   
           5.0                          1                     4   
           6.0                          3                     9   
...                                   ...                   ...   
2021-12    25.0                         3                     8   
           26.0                         4                     9   
           35.0                         4                    24   
           39.0                         2                    13   
           77.0                         0                     2   

                     DRIVER_COUNT_65_74YR  DRIVER_COUNT_75PLUS  \
year_month DIST_NUM                                              
2010-01    1.0                          1                    0   
           2.0                          4                    1   
           3.0                          6                    1   
           5.0                          1                    0   
           6.0                          2                    1   
...                                   ...                  ...   
2021-12    25.0                         3                    0   
           26.0                         3                    0   
           35.0                         4                    1   
           39.0                         1                    0   
           77.0                         0                    0   

                     ILLUMINATION_1.0  ILLUMINATION_2.0  ILLUMINATION_3.0  \
year_month DIST_NUM                                                         
2010-01    1.0                     11                 0                 6   
           2.0                     19                 0                17   
           3.0                     18                 2                21   
           5.0                      8                 0                 4   
           6.0                     20                 0                14   
...                               ...               ...               ...   
2021-12    25.0                    17                 0                18   
           26.0                    16                 0                14   
           35.0                    16                 1                26   
           39.0                    17                 0                32   
           77.0                     0                 0                 1   

                     ILLUMINATION_4.0  ILLUMINATION_5.0  ILLUMINATION_6.0  \
year_month DIST_NUM                                                         
2010-01    1.0                      0                 0                 0   
           2.0                      2                 0                 0   
           3.0                      1                 0                 1   
           5.0                      0                 0                 0   
           6.0                      1                 2                 0   
...                               ...               ...               ...   
2021-12    25.0                     0                 1                 2   
           26.0                     1                 0                 1   
           35.0                     2                 0                 3   
           39.0                     2                 2                 1   
           77.0                     0                 0                 0   

                     ...  WEATHER1_3  WEATHER1_4  WEATHER1_5  WEATHER1_6  \
year_month DIST_NUM  ...                                                   
2010-01    1.0       ...          14           0           0           0   
           2.0       ...          32           0          

In [142]:
#we conduct the transformation of log(1+x)
crash_useful_feat_transformed = np.log1p(crash_useful_feat)

In [143]:
crash_useful_feat_transformed

DRIVER_COUNT_16_20YR  DRIVER_COUNT_50_64YR  \
year_month DIST_NUM                                               
2010-01    1.0                   0.693147              1.791759   
           2.0                   1.609438              2.197225   
           3.0                   1.098612              2.302585   
           5.0                   0.693147              1.609438   
           6.0                   1.386294              2.302585   
...                                   ...                   ...   
2021-12    25.0                  1.386294              2.197225   
           26.0                  1.609438              2.302585   
           35.0                  1.609438              3.218876   
           39.0                  1.098612              2.639057   
           77.0                  0.000000              1.098612   

                     DRIVER_COUNT_65_74YR  DRIVER_COUNT_75PLUS  \
year_month DIST_NUM                                              
2010-01    1.0                   0.693147             0.000000   
           2.0                   1.609438             0.693147   
           3.0                   1.945910             0.693147   
           5.0                   0.693147             0.000000   
           6.0                   1.098612             0.693147   
...                                   ...                  ...   
2021-12    25.0                  1.386294             0.000000   
           26.0                  1.386294             0.000000   
           35.0                  1.609438             0.693147   
           39.0                  0.693147             0.000000   
           77.0                  0.000000             0.000000   

                     ILLUMINATION_1.0  ILLUMINATION_2.0  ILLUMINATION_3.0  \
year_month DIST_NUM                                                         
2010-01    1.0               2.484907          0.000000          1.945910   
           2.0               2.995732          0.000000          2.890372   
           3.0               2.944439          1.098612          3.091042   
           5.0               2.197225          0.000000          1.609438   
           6.0               3.044522          0.000000          2.708050   
...                               ...               ...               ...   
2021-12    25.0              2.890372          0.000000          2.944439   
           26.0              2.833213          0.000000          2.708050   
           35.0              2.833213          0.693147          3.295837   
           39.0              2.890372          0.000000          3.496508   
           77.0              0.000000          0.000000          0.693147   

                     ILLUMINATION_4.0  ILLUMINATION_5.0  ILLUMINATION_6.0  \
year_month DIST_NUM                                                         
2010-01    1.0               0.000000          0.000000          0.000000   
           2.0               1.098612          0.000000          0.000000   
           3.0               0.693147          0.000000          0.693147   
           5.0               0.000000          0.000000          0.000000   
           6.0               0.693147          1.098612          0.000000   
...                               ...               ...               ...   
2021-12    25.0              0.000000          0.693147          1.098612   
           26.0              0.693147          0.000000          0.693147   
           35.0              1.098612          0.000000          1.386294   
           39.0              1.098612          1.098612          0.693147   
           77.0              0.000000          0.000000          0.000000   

                     ...  WEATHER1_3  WEATHER1_4  WEATHER1_5  WEATHER1_6  \
year_month DIST_NUM  ...                                                   
2010-01    1.0       ...    2.708050    0.000000    0.000000    0.000000   
           2.0       ...    3.496508    0.000000    0.0000

## Deal with traffic_stops data

In [87]:
traffic_stops.head()

,objectid,date,time,location,lat,lng,district,service_area,subject_age,subject_race,...,contraband_found,frisk_performed,search_conducted,search_person,search_vehicle,raw_race,raw_individual_contraband,raw_vehicle_contraband,fips,year_month
0,1,2014-01-01,01:14:00,NaN,NaN,NaN,19.0,191,31.0,black,...,True,False,True,True,False,Black - Non-Latino,True,False,NaN,2014-01
1,2,2014-01-01,01:57:00,NaN,NaN,NaN,12.0,121,21.0,black,...,False,True,True,True,False,Black - Non-Latino,False,False,NaN,2014-01
2,3,2014-01-01,03:30:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,24.0,black,...,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11,2014-01
3,4,2014-01-01,03:40:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,20.0,black,...,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11,2014-01
4,5,2014-01-01,08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19.0,193,31.0,black,...,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210101e+11,2014-01


In [102]:
# cat feat: subject race, subject sex, type, arrest_made, outcome, contraband_found, frisk_performed, search_conducted

traffic_stop_useful_cat_feat = traffic_stops[["date","district","subject_race", "subject_sex", "type", "arrest_made", "outcome", "contraband_found", "frisk_performed", "search_conducted"]]
traffic_stop_useful_cat_feat["outcome"] = traffic_stop_useful_cat_feat['outcome'].fillna("none")
traffic_stop_useful_cat_feat["contraband_found"] = traffic_stop_useful_cat_feat['contraband_found'].fillna(False).astype(bool)
traffic_stop_useful_cat_feat

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,date,district,subject_race,subject_sex,type,arrest_made,outcome,contraband_found,frisk_performed,search_conducted
0,2014-01-01,19.0,black,male,pedestrian,True,arrest,True,False,True
1,2014-01-01,12.0,black,male,pedestrian,True,arrest,False,True,True
2,2014-01-01,18.0,black,male,pedestrian,False,none,False,False,False
3,2014-01-01,18.0,black,male,pedestrian,False,none,False,False,False
4,2014-01-01,19.0,black,male,vehicular,False,none,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1865091,2018-04-14,12.0,black,male,pedestrian,False,none,False,False,False
1865092,2018-04-14,39.0,asian/pacific islander,male,pedestrian,False,none,False,False,False
1865093,2018-04-14,35.0,black,male,vehicular,True,arrest,False,True,True
1865094,2018-04-14,35.0,black,male,vehicular,True,arrest,False,True,True


In [105]:
#split traffic_stops into different year_month units
traffic_stop_useful_cat_feat['date'] = pd.to_datetime(traffic_stop_useful_cat_feat['date'])
traffic_stop_useful_cat_feat['year_month'] = traffic_stop_useful_cat_feat['date'].dt.to_period('M')

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [106]:
number_of_traffic_stops = pd.get_dummies(traffic_stop_useful_cat_feat).groupby(["year_month","district"]).count()['arrest_made']
number_of_traffic_stops

year_month  district
2014-01     1.0          382
            2.0          304
            3.0          176
            5.0           47
            6.0          147
                        ... 
2018-04     25.0         947
            26.0         293
            35.0        1364
            39.0        1070
            77.0          35
Name: arrest_made, Length: 1144, dtype: int64

In [149]:
traffic_stop_sum = pd.get_dummies(traffic_stop_useful_cat_feat).groupby(["year_month","district"]).sum()
traffic_stop_sum.columns = ['traffic_stop_'+i+'_count' for i in traffic_stop_sum.columns]
traffic_stop_sum['traffic_stop_total'] = number_of_traffic_stops
traffic_stop_sum

traffic_stop_arrest_made_count  \
year_month district                                   
2014-01    1.0                                   12   
           2.0                                   19   
           3.0                                    4   
           5.0                                    0   
           6.0                                   14   
...                                             ...   
2018-04    25.0                                  39   
           26.0                                  21   
           35.0                                  70   
           39.0                                  49   
           77.0                                   5   

                     traffic_stop_contraband_found_count  \
year_month district                                        
2014-01    1.0                                         2   
           2.0                                         5   
           3.0                                         0   
           5.0                                         1   
           6.0                                         8   
...                                                  ...   
2018-04    25.0                                       16   
           26.0                                        6   
           35.0                                       19   
           39.0                                       18   
           77.0                                        2   

                     traffic_stop_frisk_performed_count  \
year_month district                                       
2014-01    1.0                                       23   
           2.0                                       28   
           3.0                                       11   
           5.0                                        2   
           6.0                                       13   
...                                                 ...   
2018-04    25.0                                     100   
           26.0                                       9   
           35.0                                     144   
           39.0                                     111   
           77.0                                       2   

                     traffic_stop_search_conducted_count  \
year_month district                                        
2014-01    1.0                                        14   
           2.0                                        12   
           3.0                                         4   
           5.0                                         2   
           6.0                                        14   
...                                                  ...   
2018-04    25.0                                       86   
           26.0                                       29   
           35.0                                      161   
           39.0                                       78   
           77.0                                       32   

                     traffic_stop_subject_race_asian/pacific islander_count  \
year_month district                                                           
2014-01    1.0                                                    28.0        
           2.0                                                    17.0        
           3.0                                                    17.0        
           5.0                                                     0.0        
           6.0                                                    15.0        
...                                                                ...        
2018-04    25.0                                                    5.0        
           26.0                                                    9.0        
           35.0                                                   23.0        
           39.0                                                    7.0        
           77.0            

In [118]:
#we conduct the transformation of log(1+x)
traffic_stop_transformed=np.log1p(traffic_stop_sum)
traffic_stop_transformed

traffic_stop_arrest_made_count  \
year_month district                                   
2014-01    1.0                             2.564949   
           2.0                             2.995732   
           3.0                             1.609438   
           5.0                             0.000000   
           6.0                             2.708050   
...                                             ...   
2018-04    25.0                            3.688879   
           26.0                            3.091042   
           35.0                            4.262680   
           39.0                            3.912023   
           77.0                            1.791759   

                     traffic_stop_frisk_performed_count  \
year_month district                                       
2014-01    1.0                                 3.178054   
           2.0                                 3.367296   
           3.0                                 2.484907   
           5.0                                 1.098612   
           6.0                                 2.639057   
...                                                 ...   
2018-04    25.0                                4.615121   
           26.0                                2.302585   
           35.0                                4.976734   
           39.0                                4.718499   
           77.0                                1.098612   

                     traffic_stop_search_conducted_count  \
year_month district                                        
2014-01    1.0                                  2.708050   
           2.0                                  2.564949   
           3.0                                  1.609438   
           5.0                                  1.098612   
           6.0                                  2.708050   
...                                                  ...   
2018-04    25.0                                 4.465908   
           26.0                                 3.401197   
           35.0                                 5.087596   
           39.0                                 4.369448   
           77.0                                 3.496508   

                     traffic_stop_subject_race_asian/pacific islander_count  \
year_month district                                                           
2014-01    1.0                                                3.367296        
           2.0                                                2.890372        
           3.0                                                2.890372        
           5.0                                                0.000000        
           6.0                                                2.772589        
...                                                                ...        
2018-04    25.0                                               1.791759        
           26.0                                               2.302585        
           35.0                                               3.178054        
           39.0                                               2.079442        
           77.0                                               1.098612        

                     traffic_stop_subject_race_black_count  \
year_month district                                          
2014-01    1.0                                    5.225747   
           2.0                                    4.990433   
           3.0                                    3.806662   
           5.0                                    2.397895   
           6.0                                    4.276666   
...                                                    ...   
2018-04    25.0                                   5.963579   
           26.0                                   4.653960   
           35.0                                   7.051856   
           39.0                               

### Merge

In [150]:
df_merged1 = pd.merge(crime_transformed.reset_index(), crash_useful_feat_transformed.reset_index(), on=['year_month','DIST_NUM'])
df_merged=pd.merge(df_merged1,traffic_stop_transformed.reset_index().rename(columns={'district': 'DIST_NUM'}), on=['year_month','DIST_NUM'])
df_merged
#here are less rows than traffic_stop_standardized dataset above, which is weired

,year_month,DIST_NUM,crime_Aggravated Assault Firearm,crime_Aggravated Assault No Firearm,crime_All Other Offenses,crime_Arson,crime_Burglary Non-Residential,crime_Burglary Residential,crime_DRIVING UNDER THE INFLUENCE,crime_Disorderly Conduct,...,traffic_stop_subject_race_unknown_count,traffic_stop_subject_race_white_count,traffic_stop_subject_sex_female_count,traffic_stop_subject_sex_male_count,traffic_stop_type_pedestrian_count,traffic_stop_type_vehicular_count,traffic_stop_outcome_arrest_count,traffic_stop_contraband_found_False_count,traffic_stop_contraband_found_True_count,traffic_stop_total
0,2014-01,1.0,0.526589,1.194706,1.457646,0.000000,0.526589,1.223156,1.124748,0.869742,...,1.609438,5.062595,4.574711,5.659482,4.990433,5.468060,2.564949,2.564949,1.098612,5.948035
1,2014-01,2.0,0.741276,1.223156,1.821607,0.526589,0.741276,1.646667,1.310506,1.310506,...,0.000000,4.624973,4.248495,5.463832,4.804021,5.214936,2.995732,2.079442,1.791759,5.720312
2,2014-01,3.0,0.741276,1.457646,1.628131,0.000000,0.959135,1.397363,1.385227,1.248441,...,0.000000,4.682131,3.828641,4.882802,4.110874,4.762174,1.609438,1.609438,0.000000,5.176150
3,2014-01,5.0,0.526589,1.124748,1.327761,0.000000,0.869742,1.248441,0.959135,1.124748,...,0.693147,3.465736,2.639057,3.555348,2.564949,3.583519,0.000000,0.693147,0.693147,3.871201
4,2014-01,6.0,0.526589,1.248441,1.610903,0.000000,0.869742,1.223156,1.162283,1.429846,...,0.693147,3.850148,3.637586,4.709530,3.891820,4.605170,2.708050,1.945910,2.197225,4.997212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,2018-04,24.0,1.327761,1.555545,1.796820,0.526589,1.124748,1.429846,1.419607,0.959135,...,2.944439,5.855072,5.347108,6.525030,5.883322,6.278521,4.007333,4.369448,2.833213,6.792344
1098,2018-04,25.0,1.474144,1.595709,1.706721,0.959135,0.959135,1.327761,1.327761,0.000000,...,3.367296,5.170484,5.476464,6.563856,5.323010,6.612041,3.688879,4.262680,2.833213,6.854355
1099,2018-04,26.0,0.741276,1.358505,1.631370,0.526589,1.080418,1.310506,0.959135,0.000000,...,1.098612,4.663439,4.304065,5.398163,4.094345,5.459586,3.091042,3.178054,1.945910,5.683580
1100,2018-04,35.0,1.439569,1.539779,1.757132,0.959135,1.080418,1.439569,1.429846,0.000000,...,2.772589,4.276666,5.638355,6.988413,5.620401,6.993933,4.262680,4.962845,2.995732,7.218910


In [151]:
df_merged.to_csv("merged_data.csv")

In [146]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1102 entries, 0 to 1101
Columns: 112 entries, year_month to traffic_stop_total
dtypes: float64(111), period[M](1)
memory usage: 972.9 KB


In [116]:
df_merged.head()

,year_month,DIST_NUM,crime_Aggravated Assault Firearm,crime_Aggravated Assault No Firearm,crime_All Other Offenses,crime_Arson,crime_Burglary Non-Residential,crime_Burglary Residential,crime_DRIVING UNDER THE INFLUENCE,crime_Disorderly Conduct,...,traffic_stop_subject_race_unknown_mean,traffic_stop_subject_race_white_mean,traffic_stop_subject_sex_female_mean,traffic_stop_subject_sex_male_mean,traffic_stop_type_pedestrian_mean,traffic_stop_type_vehicular_mean,traffic_stop_outcome_arrest_mean,traffic_stop_contraband_found_False_mean,traffic_stop_contraband_found_True_mean,traffic_stop_total
0,2014-01,1.0,0.526589,1.194706,1.457646,0.000000,0.526589,1.223156,1.124748,0.869742,...,1.609438,5.062595,4.574711,5.659482,4.990433,5.468060,2.564949,2.564949,1.098612,5.948035
1,2014-01,2.0,0.741276,1.223156,1.821607,0.526589,0.741276,1.646667,1.310506,1.310506,...,0.000000,4.624973,4.248495,5.463832,4.804021,5.214936,2.995732,2.079442,1.791759,5.720312
2,2014-01,3.0,0.741276,1.457646,1.628131,0.000000,0.959135,1.397363,1.385227,1.248441,...,0.000000,4.682131,3.828641,4.882802,4.110874,4.762174,1.609438,1.609438,0.000000,5.176150
3,2014-01,5.0,0.526589,1.124748,1.327761,0.000000,0.869742,1.248441,0.959135,1.124748,...,0.693147,3.465736,2.639057,3.555348,2.564949,3.583519,0.000000,0.693147,0.693147,3.871201
4,2014-01,6.0,0.526589,1.248441,1.610903,0.000000,0.869742,1.223156,1.162283,1.429846,...,0.693147,3.850148,3.637586,4.709530,3.891820,4.605170,2.708050,1.945910,2.197225,4.997212
